In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model, Sequential
#from keras.layers import Input
from tensorflow.keras.layers import Dense, Flatten, Conv2D
import matplotlib.pyplot as plt
import pandas as pd 
import seaborn as sns

import numpy as np
import os
import cv2

classes = ["Neutrophil", "Metamyelocyte", "Myelocyte", "Promyelocyte", "Blast", "Erythroblast",
           "Megakaryocyte_nucleus", "Lymphocyte", "Monocyte", "Plasma_cell", "Eosinophil",
           "Basophil", "Megakaryocyte", "Other_cell", "Debris", "Histiocyte", "Platelet",
           "Platelet_clump", "Mast_cell"]

In [3]:
tf.__version__

'2.2.0'

# load the model feature extractor model.

### I couldn't use tf > 2.2 due to the following error:
"Op type not registered 'Addons>Mish' in binary running on DESKTOP-XXXXX. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed."

## The feature extractor backbone of YOLO was saved into a Keras model

In [4]:
import tensorflow_addons as tfa
tfa.register_all()

model_path = "C:\\Users\\taher\\yolov4_weight\\model_x.h5"
cpsdn53_model = keras.models.load_model(model_path)

# Image preprocessing

In [5]:
from PIL import Image

def preprocess(image_path):

    image = Image.open(image_path)
    image = np.array(image)
    resized_image = tf.image.resize(image, (608, 608))
    resized_image = np.array(resized_image)
    resized_image = resized_image / 255
    input_data = resized_image[np.newaxis, ...].astype(np.float32)
    
    return input_data

In [ ]:
predictions_directory = "...."
predictions = os.listdir(predictions_directory)

In [ ]:
from IPython.display import clear_output
from PIL import Image


for c in classes: 
    newpath = "cells\\" + c 
    if not os.path.exists(newpath):
        os.makedirs(newpath)
        
newpath = "dfs\\" 
if not os.path.exists(newpath):
    os.makedirs(newpath)

for prediction in predictions:
    
    yolo_pred = pd.read_csv(os.path.join(predictions_directory, prediction), index_col=0)
    
    image_file = prediction.replace("csv", "png")
    
    image = cv2.imread(image_file)
    input_data = preprocess(image)

    features = cpsdn53_model.predict(input_data)
    
    cell_dfs = []
    cell_infos = []
    
    for i in range(yolo_pred.shape[0]):
        
        xx = int(yolo_pred["x"].iloc[i] * 512)
        ww = int(yolo_pred["width"].iloc[i] * 512/2)

        yy = int(yolo_pred["y"].iloc[i] * 512)
        hh = int(yolo_pred["height"].iloc[i] * 512/2)
        
        #remove cropped cells at the edges
        periphery = False
        if xx - ww < 5:
            periphery = True
        if xx + ww > 507:
            periphery = True
        if yy - hh < 5:
            periphery = True
        if yy + hh > 507:
            periphery = True
        
        if periphery == False:
            
            cell = image[yy-hh:yy+hh, xx-ww:xx+ww,:]
            cell = Image.fromarray(cell)

            cell_loc = "cells" + "\\" + classes[int(yolo_pred["type"].iloc[i])] + "\\" + image_file[:-4] + "_" + str(i) + ".png"

            cell.save(cell_loc)

            cords = np.array([xx, ww, yy, hh]) * 608/512
            cords = cords/8
            cords = cords.astype(int)

            if cords[1] == 0:
                cords[1] = 1

            if cords[3] == 0:
                cords[3] = 1

            f = features[0][0][cords[2] - cords[3]:cords[2] + cords[3], cords[0] - cords[1]:cords[0] + cords[1], :]
            f_x = np.mean(f, axis = (0,1))
            f_x = np.append(f_x, np.max(f, axis = (0,1)))
            
            cell_type = classes[int(yolo_pred["type"].iloc[i])]
            cell_index = i 
            
            cell_infos.append([image_file, cell_index, cell_type])
            cell_dfs.append(f_x)
        
        try:   
            cell_pd = pd.DataFrame(cell_dfs)
            info_pd = pd.DataFrame(cell_infos, columns=["image_file", "cell_index", "cell_type"])
            cell_pd = pd.concat([info_pd ,cell_pd], axis=1)
            
            cell_pd.to_csv("dfs\\" + image_file.replace("png", "csv"))
        except:
            pass
        
        
    clear_output()        
    progress = (predictions.index(prediction) + 1)/len(predictions)
    progress = int(progress * 100)
    progress_bar = "|" * progress
    print(progress_bar, progress)
    
dfs = []
for df in os.listdir("dfs\\"):
    dfs.append(pd.read_csv("dfs\\" + df, index_col=0))
    
df_slide = pd.concat(dfs).reset_index(drop = True)

df_slide.to_pickle("all_deep_features.pkl")
